<a href="https://colab.research.google.com/github/PRAKASHMS7/Phishing-Detection-By-Using-ML-Models/blob/main/Wrapper_Based_Approaches/WRAPPER_BASED_TECHNIQUES_ipy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**FORWARD SELECTION**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.impute import SimpleImputer
# Load the dataset
file_path = '/content/All.csv'
df = pd.read_csv(file_path)



# Load dataset
df = pd.read_csv("All.csv")

# Separate features and target
X = df.drop(columns=['URL_Type_obf_Type'])
y = df['URL_Type_obf_Type']

# Clean data: Replace inf and very large values with NaN
X.replace([np.inf, -np.inf], np.nan, inplace=True)
X = X.applymap(lambda x: np.nan if isinstance(x, float) and (x > 1e308 or x < -1e308) else x)

# Impute missing values using column mean
imputer = SimpleImputer(strategy='mean')
X_imputed = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)

# Encode target labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Take a sample for faster computation
X_sample, _, y_sample, _ = train_test_split(X_imputed, y_encoded, train_size=0.3, stratify=y_encoded, random_state=42)

# Define classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)

# Forward Feature Selection (Top 40)
sfs = SequentialFeatureSelector(clf, n_features_to_select=40, direction='forward', scoring='accuracy', cv=3, n_jobs=-1)
sfs.fit(X_sample, y_sample)

# Get selected features
selected_features = X_sample.columns[sfs.get_support()].tolist()

# Train-test split with selected features
X_train, X_test, y_train, y_test = train_test_split(X_sample[selected_features], y_sample, test_size=0.3, random_state=42, stratify=y_sample)

# Train and predict
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
y_prob = clf.predict_proba(X_test)

# Evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)
roc_auc = roc_auc_score(y_test, y_prob, multi_class='ovr')

# Display results
print("\nTop 40 Selected Features:\n")
for i, feat in enumerate(selected_features, 1):
    print(f"{i}. {feat}")

print("\nEvaluation Metrics on Selected Features:")
print(f"Accuracy:  {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1 Score:  {f1:.4f}")
print(f"ROC AUC:   {roc_auc:.4f}")



<ipython-input-2-ff718aa33814>:24: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  X = X.applymap(lambda x: np.nan if isinstance(x, float) and (x > 1e308 or x < -1e308) else x)



Top 40 Selected Features:

1. domain_token_count
2. avgdomaintokenlen
3. longdomaintokenlen
4. ldl_url
5. ldl_domain
6. ldl_getArg
7. dld_domain
8. dld_getArg
9. domainlength
10. fileNameLen
11. this.fileExtLen
12. ArgUrlRatio
13. argDomanRatio
14. executable
15. isPortEighty
16. NumberofDotsinURL
17. ISIpAddressInDomainName
18. CharacterContinuityRate
19. URL_DigitCount
20. host_DigitCount
21. Directory_DigitCount
22. File_name_DigitCount
23. Extension_DigitCount
24. host_letter_count
25. Directory_LetterCount
26. Filename_LetterCount
27. Extension_LetterCount
28. Domain_LongestWordLength
29. Path_LongestWordLength
30. Arguments_LongestWordLength
31. URLQueries_variable
32. spcharUrl
33. delimeter_Domain
34. delimeter_path
35. NumberRate_URL
36. NumberRate_Domain
37. NumberRate_DirectoryName
38. SymbolCount_Domain
39. SymbolCount_Directoryname
40. Entropy_Domain

Evaluation Metrics on Selected Features:
Accuracy:  0.9564
Precision: 0.9574
Recall:    0.9564
F1 Score:  0.9566
ROC AUC: 

**BACKWARD SELECTION**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.impute import SimpleImputer
import time

# Load dataset
df = pd.read_csv("All.csv")

# Separate features and target
X = df.drop(columns=['URL_Type_obf_Type'])
y = df['URL_Type_obf_Type']

# Clean data: Replace inf and very large values with NaN
X.replace([np.inf, -np.inf], np.nan, inplace=True)

# Update deprecated applymap with column-wise map
for col in X.columns:
    X[col] = X[col].map(lambda x: np.nan if isinstance(x, float) and (x > 1e308 or x < -1e308) else x)

# Impute missing values using column mean
imputer = SimpleImputer(strategy='mean')
X_imputed = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)

# Encode target labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Take a smaller sample for faster computation
X_sample, _, y_sample, _ = train_test_split(X_imputed, y_encoded, train_size=0.2, stratify=y_encoded, random_state=42)

# Define classifier with optimized parameters for faster execution
clf = RandomForestClassifier(
    n_estimators=50,
    max_depth=10,
    min_samples_split=5,
    random_state=42,
    n_jobs=-1,
    verbose=1
)

# Backward Feature Selection with optimizations
start_time = time.time()

# First reduce features using feature importance
clf.fit(X_sample, y_sample)
importances = clf.feature_importances_
top_60_features = X_sample.columns[np.argsort(importances)[-60:]]
X_reduced = X_sample[top_60_features]

# Perform backward selection on reduced set
sfs_backward = SequentialFeatureSelector(
    clf,
    n_features_to_select=40,
    direction='backward',
    scoring='accuracy',
    cv=2,
    n_jobs=-1
)

sfs_backward.fit(X_reduced, y_sample)
selected_features = X_reduced.columns[sfs_backward.get_support()].tolist()

print(f"\nBackward selection completed in {(time.time() - start_time)/60:.2f} minutes")

# Train-test split with selected features
X_train, X_test, y_train, y_test = train_test_split(
    X_sample[selected_features], y_sample,
    test_size=0.3,
    random_state=42,
    stratify=y_sample
)

# Train and predict with final classifier
final_clf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
final_clf.fit(X_train, y_train)
y_pred = final_clf.predict(X_test)
y_prob = final_clf.predict_proba(X_test)

# Evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)
roc_auc = roc_auc_score(y_test, y_prob, multi_class='ovr')

# Display results
print("\nTop 40 Selected Features (Backward Selection):\n")
for i, feat in enumerate(selected_features, 1):
    print(f"{i}. {feat}")

print("\nEvaluation Metrics on Selected Features:")
print(f"Accuracy:  {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1 Score:  {f1:.4f}")
print(f"ROC AUC:   {roc_auc:.4f}")


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    1.4s finished



Backward selection completed in 16.69 minutes

Top 40 Selected Features (Backward Selection):

1. dld_url
2. Directory_DigitCount
3. charcompvowels
4. Entropy_URL
5. LongestVariableValue
6. Entropy_Extension
7. File_name_DigitCount
8. Filename_LetterCount
9. path_token_count
10. host_DigitCount
11. fileNameLen
12. URL_DigitCount
13. Querylength
14. NumberRate_AfterPath
15. SymbolCount_FileName
16. Query_LetterCount
17. ldl_url
18. argPathRatio
19. Query_DigitCount
20. longdomaintokenlen
21. Domain_LongestWordLength
22. Extension_LetterCount
23. LongestPathTokenLength
24. delimeter_path
25. URLQueries_variable
26. NumberRate_Extension
27. NumberRate_URL
28. domain_token_count
29. argDomanRatio
30. CharacterContinuityRate
31. SymbolCount_Directoryname
32. Entropy_Domain
33. spcharUrl
34. avgpathtokenlen
35. avgdomaintokenlen
36. domainlength
37. host_letter_count
38. NumberRate_FileName
39. ArgUrlRatio
40. tld

Evaluation Metrics on Selected Features:
Accuracy:  0.9514
Precision: 0.9522

**EXHAUSTIVE SELECTION**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.feature_selection import RFE
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report

def load_and_preprocess(filepath):
    """Optimized data loading and preprocessing"""
    data = pd.read_csv(filepath)

    # Handle missing values
    data = data.fillna(data.median(numeric_only=True))

    # Encode target
    le = LabelEncoder()
    data['URL_Type_obf_Type'] = le.fit_transform(data['URL_Type_obf_Type'])

    # Separate features and target
    X = data.select_dtypes(include=[np.number]).drop(columns=['URL_Type_obf_Type'], errors='ignore')
    y = data['URL_Type_obf_Type']

    # Handle infinite values
    X = X.replace([np.inf, -np.inf], np.nan).fillna(X.median())

    return X, y, le

def main():
    # Load and preprocess
    print("Loading and preprocessing data...")
    X, y, le = load_and_preprocess("/content/All.csv")

    # Standardize
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Split data
    X_train, X_test, y_train, y_test = train_test_split(
        X_scaled, y, test_size=0.2, random_state=42, stratify=y
    )

    # Initialize model
    model = LogisticRegression(max_iter=1000, solver='liblinear', n_jobs=-1)

    # Faster feature selection using RFE
    print("\nPerforming feature selection (RFE)...")
    selector = RFE(model, n_features_to_select=40, step=0.1, verbose=1)
    selector.fit(X_train, y_train)

    # Get selected features
    selected_features = X.columns[selector.support_]

    print("\n=== TOP 40 SELECTED FEATURES ===")
    for i, feat in enumerate(selected_features, 1):
        print(f"{i:2d}. {feat}")

    # Train on selected features
    X_train_selected = selector.transform(X_train)
    X_test_selected = selector.transform(X_test)

    model.fit(X_train_selected, y_train)

    # Evaluate
    y_pred = model.predict(X_test_selected)
    y_proba = model.predict_proba(X_test_selected)

    print("\n=== MODEL PERFORMANCE ===")
    print(f"Accuracy:  {accuracy_score(y_test, y_pred):.4f}")
    print(f"Precision: {precision_score(y_test, y_pred, average='weighted'):.4f}")
    print(f"Recall:    {recall_score(y_test, y_pred, average='weighted'):.4f}")
    print(f"F1 Score:  {f1_score(y_test, y_pred, average='weighted'):.4f}")
    print(f"ROC AUC:   {roc_auc_score(y_test, y_proba, multi_class='ovr'):.4f}")

    print("\n=== CLASSIFICATION REPORT ===")
    print(classification_report(y_test, y_pred, target_names=le.classes_))

if __name__ == "__main__":
    main()

Loading and preprocessing data...

Performing feature selection (RFE)...
Fitting estimator with 79 features.


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1271: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 2.
  warnings.warn(


Fitting estimator with 72 features.


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1271: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 2.
  warnings.warn(


Fitting estimator with 65 features.


...

Fitting estimator with 44 features.


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1271: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 2.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1271: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 2.
  warnings.warn(



=== TOP 40 SELECTED FEATURES ===
 1. Querylength
 2. domain_token_count
 3. path_token_count
 4. avgdomaintokenlen
 5. longdomaintokenlen
 6. tld
 7. charcompvowels
 8. ldl_url
 9. ldl_path
10. ldl_getArg
11. dld_url
12. dld_getArg
13. urlLen
14. domainlength
15. pathLength
16. subDirLen
17. ArgLen
18. pathurlRatio
19. ArgUrlRatio
20. argDomanRatio
21. domainUrlRatio
22. pathDomainRatio
23. argPathRatio
24. LongestVariableValue
25. URL_DigitCount
26. host_DigitCount
27. Extension_DigitCount
28. Query_DigitCount
29. URL_Letter_Count
30. host_letter_count
31. Extension_LetterCount
32. Query_LetterCount
33. LongestPathTokenLength
34. URLQueries_variable
35. delimeter_path
36. NumberRate_Domain
37. NumberRate_AfterPath
38. SymbolCount_URL
39. SymbolCount_Domain
40. SymbolCount_Afterpath


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1271: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 2.
  warnings.warn(



=== MODEL PERFORMANCE ===
Accuracy:  0.7996
Precision: 0.8050
Recall:    0.7996
F1 Score:  0.7976
ROC AUC:   0.9564

=== CLASSIFICATION REPORT ===
              precision    recall  f1-score   support

  Defacement       0.82      0.79      0.81      1586
      benign       0.78      0.87      0.82      1556
     malware       0.82      0.61      0.70      1343
    phishing       0.72      0.85      0.78      1517
        spam       0.89      0.86      0.87      1340

    accuracy                           0.80      7342
   macro avg       0.81      0.80      0.80      7342
weighted avg       0.81      0.80      0.80      7342



**RECURSIVE SELECTION**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import (accuracy_score, precision_score,
                           recall_score, f1_score, roc_auc_score,
                           classification_report)

# Load data
data = pd.read_csv('/content/All.csv')

# Handle infinite and missing values
data.replace([np.inf, -np.inf], np.nan, inplace=True)  # Convert inf to NaN
data.dropna(inplace=True)  # Remove rows with any NaN values

# Verify no infinite or NaN values remain
print("Infinity check:", np.isinf(data.select_dtypes(include=np.number)).any().any())
print("NaN check:", data.isna().any().any())

# Prepare features/target
X = data.drop(columns='URL_Type_obf_Type')
y = data['URL_Type_obf_Type']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Get feature importance
model = RandomForestClassifier(n_estimators=200, random_state=42, n_jobs=-1)
model.fit(X_train, y_train)  # This should now work

# Select top 40 features
importance = pd.DataFrame({'Feature': X.columns, 'Importance': model.feature_importances_})
top_40 = importance.nlargest(40, 'Importance')

# Display features
print("\nTop 40 Features by Importance:")
print(top_40.to_string(index=False))

# Train model with selected features
model.fit(X_train[top_40['Feature']], y_train)
y_pred = model.predict(X_test[top_40['Feature']])
y_proba = model.predict_proba(X_test[top_40['Feature']])

# Compute metrics
print("\nModel Metrics:")
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print(f"Precision: {precision_score(y_test, y_pred, average='weighted'):.4f}")
print(f"Recall: {recall_score(y_test, y_pred, average='weighted'):.4f}")
print(f"F1 Score: {f1_score(y_test, y_pred, average='weighted'):.4f}")
print(f"ROC AUC: {roc_auc_score(y_test, y_proba, multi_class='ovr'):.4f}")

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

Infinity check: False
NaN check: False

Top 40 Features by Importance:
                  Feature  Importance
        avgdomaintokenlen    0.036005
                   urlLen    0.027954
      NumberRate_FileName    0.027417
    Extension_LetterCount    0.027204
     Extension_DigitCount    0.026785
             domainlength    0.025742
       SymbolCount_Domain    0.025150
             pathurlRatio    0.024794
               pathLength    0.023207
       domain_token_count    0.022612
  CharacterContinuityRate    0.022581
              fileNameLen    0.022316
           domainUrlRatio    0.022299
                subDirLen    0.022251
                      tld    0.022210
          pathDomainRatio    0.021146
   LongestPathTokenLength    0.020505
           NumberRate_URL    0.020428
           Entropy_Domain    0.020219
            argDomanRatio    0.019985
       longdomaintokenlen    0.019727
        host_letter_count    0.018471
    Directory_LetterCount    0.018396
        NumberofD